# Lab 1 - POD analysis of separated flow past a NACA0018 airfoil 

In this lab we are going to apply POD to obtain a better understanding of the structure and evolution of the flow past a NACA0018 airfoil. In addition, this lab will teach you practical data manipulation skills using NumPy.

Data for this lab has been collected by Dr Sean Symon for his PhD (see [this link](https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/article/tale-of-two-airfoils-resolventbased-modelling-of-an-oscillator-versus-an-amplifier-from-an-experimental-mean/2CE698E25A77016C41636048BCFA17AE)). The data has been collected from Particle Image Velocimetry (PIV), a optical measurement technique (i.e. using a camera) that measures the dislacement of small smoke particles released into the flow to extract two velocity components in a plane illuminated by a LASER sheet. You can find more about PIV on its Wikipedia [article](https://en.wikipedia.org/wiki/Particle_image_velocimetry), or by taking the Experimental Methods for Aerodynamics module SESA6070. Details of the experiments can be found in the journal article referenced above (or by talking to Dr Symon).

Two datasets are available, at two angles of attack: $\alpha = 0$ and 10 degrees. We'll start by examining the second case, where large-scale flow separation is expected on the upper surface of the airfoil. You are welcome to examine the other case in your own time.

We will start with a few basic imports:

In [1]:
import matplotlib.pyplot as plt
import scipy.io
import numpy as np

%matplotlib qt

<class 'ImportError'>: Failed to import any qt binding

and also import a function from a custom module that we will use to plot the airfoil.

In [2]:
from airfoil import plot_airfoil

Now, let's start by loading the data. These are Matlab files, which we can also open using `scipy.io.loadmat`. Make sure to replace the path to suit your own file system.

In [3]:
# use this variable later too
AoA = 10

# load file
mat = scipy.io.loadmat("airfoil-PIV-data/AoA=%d.mat" % AoA)

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'airfoil-PIV-data/AoA=10.mat'

The variable `mat` is simply a normal Python dictionary that contains a bunch of things. 

In [4]:
mat

<class 'NameError'>: name 'mat' is not defined

Most of the values in the dictionary are NumPy arrays with data. The most important variables are the grid points `mat["x"]` and `mat["y"]` and the velocity data `mat["u"]` and `mat["v"]`, for the two velocity components.

The grid points are stored in 2D arrays:

In [5]:
mat["x"].shape, mat["y"].shape

<class 'NameError'>: name 'mat' is not defined

and we have `nx=176` points along the `x` axis (the streamwise direction) and `ny=46` points along the `y` axis (the transverse direction).

For the velocity components, we have 3D arrays:

In [6]:
mat["u"].shape, mat["v"].shape

<class 'NameError'>: name 'mat' is not defined

where the first index is associated with time. More specifically, we have 3400 snapshots so `mat["u"][0, :, :]` will be the `u` velocity field at time `t=0`, `mat["u"][1, :, :]` will be the `u` velocity field at time `t=dt` and so on.

The data is time resolved and the time interval is:

In [7]:
mat["dt"]

<class 'NameError'>: name 'mat' is not defined

This is a small quantity, and shows that we need roughly 1/0.008 = 125 snapshots for a fluid particle to travel across the entire chord of the airfoil, at the external flow speed.

Note that all these kinematic quantities have been normalised using the chord $c$ and the external velocity $u_\infty$. 

Let's store some of these variables for later use.

In [8]:
m, nx, ny = mat["u"].shape

<class 'NameError'>: name 'mat' is not defined

### Task 1 - Visualising the data

We'll start by plotting one snapshot, so you can learn how to visualise this dataset. We'll plot the $u$ velocity component, using the `contourf` function and also plot vectors to visualise the velocity field using `quiver`. Here is how to do it.

In [9]:
plt.figure(0)
plt.clf()

# plot contours from -1.5 to 1.5
levels = np.linspace(-1.5, 1.5, 20)
plt.contourf(mat["x"], mat["y"], mat["u"][10], levels, cmap="seismic")

# add colorbar
plt.colorbar(shrink=0.25, ticks=[-1.5, 0, 1.5], label="$\overline{u}/u_\infty$")

# only plot one vector every three, to avoid crowding
skip = 1
plt.quiver(mat["x"][::skip, ::skip], 
           mat["y"][::skip, ::skip], 
           mat["u"][0][::skip, ::skip], 
           mat["v"][0][::skip, ::skip])

# must set the aspect ratio to 1 for the x and y axis to have the same metric
plt.gca().set_aspect(1)

# always add labels
plt.xlabel("x/c")
plt.ylabel("y/c")

# also plot the airfoil
plot_airfoil(AoA)

<class 'ImportError'>: Failed to import any qt binding

Note all the elements of the plot (colorbar, labels, etc) and make sure you use a similar style for your coursework. Note also the white area near the airfoil: this is where we do not have data and the array simply contain a bunch of zeros in this area. This is standard in experiments!

Since the data is time resolved, we can create a little movie to get a first grasp of this flow. Note that I am using `plt.pause` (this has been tested on a mac computer, but it should work on windows too). I am plotting one every 20 snapshots of the first 1000 snapshots, to ensure the video proceeds at a decent speed. I am plotting contours of the $u$ and $v$ velocities.

In [ ]:
plt.figure(1, figsize=(7, 5))
plt.subplots_adjust(top=0.96, hspace=0.1, bottom=0.05, right=0.99)
plt.clf()

levels = np.linspace(-1.5, 1.5, 30)

for it in range(0, 200, 5):
    # need to clear the figure before every loop iteration
    plt.clf()

    plt.subplot(211)
    plt.contourf(mat["x"], mat["y"], mat["u"][it], levels, cmap="seismic")
    plt.gca().set_aspect(1)
    plt.colorbar(shrink=0.5, ticks=[-1.5, 0, 1.5], label="$\\overline{u}/u_\infty$")
    plt.xlabel("x/c")
    plt.ylabel("y/c")
    plot_airfoil(AoA)

    plt.subplot(212)
    plt.contourf(mat["x"], mat["y"], mat["v"][it], levels, cmap="seismic")
    plt.gca().set_aspect(1)
    plt.colorbar(shrink=0.5, ticks=[-1.5, 0, 1.5], label="$\\overline{v}/u_\infty$")
    plt.xlabel("x/c")
    plt.ylabel("y/c")
    plot_airfoil(AoA)
    
    plt.pause(0.001)

Good, now we need to compute time averaged quantities. We'll use the function `np.mean`, noting that we pass the extra argument `axis=0` to tell numpy to average over the first dimension of the array, i.e. along the data axis. 

In [ ]:
# compute mean flow
u_bar = np.mean(mat["u"], axis=0)
v_bar = np.mean(mat["v"], axis=0)

Let's check the shape of `u_bar`:

In [ ]:
u_bar.shape

The output is a 2D array with size `nx, ny`, i.e. as any other snapshot. This is because the function `np.mean` has averaged over the first dimension of the 3D array  `mat["u"]`, squeezing the temporal dimension away. This approach is very powerful when you have data organised in arrays, like in the present case.

We should also compute turbulence quantities, e.g the Reynolds stresses. The NumPy function `np.var` does exactly what we need, i.e. it computes the averaged squared deviation from the mean. This is exactly the energy of the fluctuations that we will try to capture using POD structures. Hence, developing a feeling for where fluctuations are more energetic will helps us later on with the interpretation of the POD modes.

In [ ]:
# compute turbulent quantiies
u_var = np.var(mat["u"], axis=0)
v_var = np.var(mat["v"], axis=0)

Let's now plot these four variables.

In [ ]:
plt.figure(2, figsize=(7, 8))
plt.subplots_adjust(top=0.96, hspace=0.1, bottom=0.05, right=0.99)
plt.clf()

plt.subplot(411)
levels = np.linspace(-1.5, 1.5, 50)
plt.contourf(mat["x"], mat["y"], u_bar, levels, cmap="seismic")
plt.gca().set_aspect(1)
plt.colorbar(shrink=0.65, ticks=[-1.5, 0, 1.5], label="$\\overline{u}/u_\infty$")
plt.xlabel("x/c")
plt.ylabel("y/c")
plot_airfoil(AoA)

plt.subplot(412)
levels = np.linspace(-0.5, 0.5, 50)
plt.contourf(mat["x"], mat["y"], v_bar, levels, cmap="seismic")
plt.gca().set_aspect(1)
plt.colorbar(shrink=0.65, ticks=[-0.5, 0, 0.5], label="$\\overline{v}/u_\infty$")
plt.xlabel("x/c")
plt.ylabel("y/c")
plot_airfoil(AoA)

plt.subplot(413)
plt.contourf(mat["x"], mat["y"], u_var, 20, cmap="magma_r")
plt.gca().set_aspect(1)
plt.gca().set_aspect(1)
plt.colorbar(shrink=0.65, ticks=[0, 0.05, 0.1], label="$\\overline{u^\prime u^\prime}/u_\infty^2$")
plt.xlabel("x/c")
plt.ylabel("y/c")
plot_airfoil(AoA)

plt.subplot(414)
plt.contourf(mat["x"], mat["y"], v_var, 20, cmap="magma_r")
plt.gca().set_aspect(1)
plt.gca().set_aspect(1)
plt.colorbar(shrink=0.65, ticks=[0, 0.05, 0.1], label="$\\overline{v^\prime v^\prime}/u_\infty^2$")
plt.xlabel("x/c")
plt.ylabel("y/c")
plot_airfoil(AoA)

Can you make sense of these fields? Can you spot the wake? Does the field of the transverse velocity component make sense to you? What about the turbulence statistics? Go back and watch the animation to develop a better understanding of this flow.

### Task 2 - Data preparation

To perform POD with the SVD we need to massage the data in a format suitable for the SVD function. Each snapshot needs to be reshaped as a column vector and all snapshots have to be stacked horizontally one next to the other. In addition, the $v$ velocity component data will have to be stacked below the $u$ component data. We have $m$ snapshots of size $nx \times ny$, so the data matrix `X` will need to have size $ (2 n_x  n_y) \times m$.

To do this, you can use a combination of the functions `np.reshape`, `np.hstack` and `np.vstack`. Check their documentation online and sketch on a piece of paper how you think the data will be structured.

In [ ]:
# TASK: transform all snapshots into column vectors (reshape), pack them one next to the 
#       other (hstack) and pack the two velocity components one on top of the other (vstack)
#
# X = 
X = np.vstack( (np.hstack([np.reshape(mat["u"][i], (nx*ny, 1)) for i in range(m)]),
                np.hstack([np.reshape(mat["v"][i], (-1, 1)) for i in range(m)])) )

If your code is correct, the following assert statements must pass without raising errors.

In [ ]:
assert X.shape   == (16192, 3400)
assert X[1, 1]   == 1.0093801585989954
assert X[-1, -1] == 0.06814444533965798

The data matrix is quite large and has size `(16192, 3400)`. Let's try to plot it as if it was an image (remember what we said about the SVD in the previous weeks). 

In [ ]:
plt.figure(3)
plt.clf()

plt.imshow(X, interpolation="none", cmap="seismic");
plt.xlabel("time")
plt.ylabel("space")

Note that we have spatial information along the rows (for the $u$ and $v$ components) and temporal information along the columns. Zoom in to see the structure in the data. There is quite a bit of regular behaviour, and many columns look similar. Good! this means that the same data can be represented by a low-rank expansion, the POD! 

Let's now transform the mean flow into a column vector. Write code similar to what you have done a few cells above to reshape the mean flow components `u_bar` and `v_bar` into a column vector `x_bar`.

In [ ]:
# TASK: transform mean flow into a vector
# x_bar = 
# x_bar = np.vstack( (np.reshape(u_bar, (-1, 1)), np.reshape(v_bar, (-1, 1))) )
x_bar = np.reshape(np.mean(X, axis=1), (-1, 1))

Now we can compute the fluctuations. We can simply subtract `x_bar` from `X`, even if their shapes do not match because of a feature of NumPy called [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).

We obtain

In [ ]:
# compute fluctuation
Xp = X - x_bar

where `p` stands for "prime". Check the shape of `Xp` - does it make sense to you?

### Task 3 - Performing POD

To perform POD we need to first think about what the correct inner product is based on the data we have. PIV data is almost always available on a uniform mesh of square or rectangular cells, and each velocity vector represents the average velocity over the cell. Let's check the grid using the function `np.diff`. This can be used to compute the grid spacing, in both the `x` and `y` directions.

In [ ]:
np.diff(mat["x"], axis=0)

In [ ]:
np.diff(mat["y"], axis=1)

We see that the grid is indeed uniform with square cells with area $A_c = 0.02032577^2$. 

In [ ]:
A_c = 0.02032577**2

In this case we probably do not need to worry much about modying the SVD algorithm as all cells contribute equally to the evaluation of integrals. The practical consequence of this is that the POD modes and modal energies would be off by a constant factor. This is not so important in this case, because a) for the POD modes we care about their spatial structure, and not their amplitude and b) for the modal energies we care about their relative importance, e.g. when we plot the relative information content.

In any case, we'll use the correct algorithm, just to make some practice.

We will use the `sparse` package from `scipy`, as it contains some classes for storing a large diagonal matrix very efficiently, i.e. not using a dense matrix, but only storing the nonzero elements. You can read more about the topic [https://en.wikipedia.org/wiki/Sparse_matrix](here).

Let's import the package

In [ ]:
import scipy.sparse as ssp

and let's create the mass matrix.

In [ ]:
W = ssp.diags( np.repeat(A_c, 2*nx*ny) )

Note how the size of this matrix is twice the number of grid points, becase a snapshot vector contains both the $u$ and $v$ components. 

How would we use this? Well, let's compute the norm of one of the snapshots

In [ ]:
X[:, 0].T @ W @ X[:, 0]

For the full algorithm, we need to form the Choleski factor, and its inverse for solving for the POD modes. Again, we use `diags` and just compute the decomposition manually since `W` is diagonal.

In [ ]:
# compute factor of the Cholesky decomposition
L = ssp.diags(np.repeat(A_c, 2*nx*ny)**0.5)

# and its inverse
Linv = ssp.diags(1/np.repeat(A_c, 2*nx*ny)**0.5)

In the next cell, you should take the SVD and compute singular vectors/values. Remember that you very likely want the economy SVD.

In [ ]:
# TASK: compute the svd of an appropriate matrix
U, S, VT = np.linalg.svd(L.T @ Xp, full_matrices=False)

Running the SVD for such a large matrix takes a bit. Let's examine the shape of the outputs. Let's start with the right singular vectors `U`

In [ ]:
U.shape

The data is organised exactly as the data matrix `Xp`, i.e. we have space (and velocity components along the rows) and modal index along the columns.

In [ ]:
S

`S` is a 1D array. It will be convenient to transform this into a matrix later on. 

The array `VT` contains the right singular vectors in each of its rows. The function `np.linalg.svd` returns it transposed by default, so make sure you understand how the data in this array is structured.


Let's now construct the POD modes, temporal coefficients and modal energies. For the POD modes, note that

In [ ]:
PHI = Linv.T @ U

In [ ]:
PHI.shape

These modes should be orthogonal and have unit norm. Let's try subtracting the identity matrix from the product of the modes with themselves

In [ ]:
np.linalg.norm(PHI.T @ W @ PHI - ssp.diags(np.ones(m)))

OK!

Finally, we "unpack" the $u$ and $v$ components of the POD, simply by splitting in half the rows of `PHI`:

In [ ]:
PHI_u = PHI[:nx*ny, :]
PHI_v = PHI[nx*ny:, :]

Note how the top half is the $u$ component of the POD mode and the bottom half is the $v$ component, exactly how we packed the data few cells above here.

Let's now obtain the temporal coefficients. I will obtain them from projecting the snapshots onto the POD modes. For the projection, I am using the mass matrix, so that I computing the correct inner product:

In [ ]:
A = Xp.T @ W @ PHI

Let's compute the modal energies

In [ ]:
# get modal energies
lambdas = S**2 / m

In the following cells, you should define a function to compute the relative information content from the modal energies and then plot the modal energies and the RIC metric. Use log scales when appropriate. You might wish to use the functions `np.cumsum` and `sum`.

In [ ]:
# TASK define a function to compute the RIC
def ric(lambdas):
    """ Compute relative information content from the modal energies """
    return np.cumsum(lambdas)/sum(lambdas)

In [ ]:
fig = plt.figure(4)
plt.subplots_adjust(wspace=0.35)
plt.clf()

# array for plotting starting from 1
i_or_k = np.arange(len(lambdas)) + 1

plt.subplot(121)
plt.plot(i_or_k, lambdas, ".")
plt.ylim(1e-8, 1e-2)
plt.grid(1)
plt.xlabel("i")
plt.ylabel("$\lambda_i$")
plt.yscale("log")

plt.subplot(122)
plt.plot(i_or_k, ric(lambdas), ".")
plt.xlabel("k")
plt.ylabel("RIC(k)")
plt.grid(1)

The modal energies drop quite fast, but it takes 130 modes to capture 90% of the energy in the data. In you spare time, you should compare the modal energies of the two datasets, for angle or attack of 0 and 10 degrees. This will give you some insight into the complexity of the two flows.

### Task 4: analysing the results

We'll start by plotting contours of the first POD mode ($v$ velocity component) and also add vectors, to get an idea of the flow direction. The crucial thing is that we  need to reshape the 1d array for the POD mode into a 2D array that can be fed to `plt.contourf`. We can simply use the NumPy function `np.reshape` for this

In [ ]:
# index of the POD to plot (plot 0, 1, 6, 100)
idx = 0

# reshape into nx by ny 2D array for plotting filled contours
phi_u = np.reshape(PHI_u[:, idx], (nx, ny))
phi_v = np.reshape(PHI_v[:, idx], (nx, ny))

In [ ]:
plt.figure(5)
plt.clf()

levels = np.linspace(-5, 5, 100)

plt.contourf(mat["x"], mat["y"], phi_u, levels, cmap="seismic")

# scale controls the length of the vector (lower scale -> longer vectors)
skip = 3
plt.quiver(mat["x"][::skip, ::skip], 
           mat["y"][::skip, ::skip],
           phi_u[::skip, ::skip],
           phi_v[::skip, ::skip], scale=50)


plt.xlabel("x/c")
plt.ylabel("y/c")
plt.gca().set_aspect(1)
plt.grid(1)

plot_airfoil(AoA)

As you can see, the first POD mode looks like a wave in the streamwise direction and  captures the process of shedding vortical structures from the airfoil. Change the code above to plot the $u$ component and check if the sign of the velocities matches with the expect flow direction. Plot also mode 2. As you can see this looks very similiar to mode 1 (note Python starts at zero), but it is shifted downstream by half a wave period. This is common for periodic flow with strongly convective components, as you need two structures (a sine and a cosine) to represent any arbitrary phase.

Try also plotting other modes, e.g. modes 4 or 20. As you can see, the higher the mode index, the more complex the flow behaviour usually becomes.

We can also plot the temporal coefficients. Let's also construct an array of times.

In [ ]:
plt.figure(6)
plt.clf()
plt.subplots_adjust(wspace=0.4, hspace=0.4)

tt = np.arange(m) * mat["dt"][0][0]

# mode 1 and 2
plt.subplot(2, 1, 1)
plt.plot(tt, A[:, 0], label="$a_1(t)$")
plt.plot(tt, A[:, 1], label="$a_2(t)$")
plt.legend()
plt.grid(1)
plt.ylabel("$a_i(t)$")
plt.xlabel("$t u_\infty / c$")

# mode 3 and 4
plt.subplot(2, 1, 2)
plt.plot(tt, A[:, 9], label="$a_3(t)$")
plt.plot(tt, A[:, 10], label="$a_4(t)$")
plt.grid(1)
plt.legend()
plt.ylabel("$a_i(t)$")
plt.xlabel("$t u_\infty / c$")

As you can see, the coefficients are highly oscillatory, but they are far from being regular sine/cosine waves. The signals look modulated as if there are multiple frequency components mixed together. This is actually a property of POD, i.e. that it can mix together spatial behaviour that occurs at different frequencies. On the other hand, DMD can separate spatial structures that evolve at different frequencies, which is useful to distinguish independent flow processes.

### Task 5: low rank reconstruction

In the last task of this lab, we are going to perform a low-rank reconstruction of the velocity fluctuation, using POD modes. Think about the cat example we have done a few lectures ago, where we have reconstructed the whole image (the whole dataset) with an increasing number of left/right singular vectors.

Let's attempt a rank-two reconstruction using two POD modes. To achieve this we take the first two columns of `PHI` and multiply by the first two columns of `A`, then transpose. 

In [ ]:
# rank-k reconstruction
k = 20
Xp_r = PHI[:, :k] @ A[:, :k].T

The product of these matrices gives us a matrix with the same shape of the data matrix. We'll append `_r`, for "reconstructed" to distinguish is from the original fluctuation data `Xp`.

In [ ]:
Xp_r.shape

What should be the rank of this matrix? Let's check:

In [ ]:
# np.linalg.matrix_rank(Xp_r)

Yes, two. This is consistent with the fact that we have constructed this matrix from the product of other smaller matrices.

The matrix `Xp_r` contains the reconstructed velocity fluctuation for all available snapshots. We can make an animation of this data and compare it with the original data. Note how I am unpacking the u and v components of the reconstructed snapshot vectors and then reshaping these into 2D arrays with size compatible with our grid data.

We plot the reconstructed flow in the top panel and the original data in the bottom one. We'll be plotting the transverse velocity component $v$ which is more indicative of large scale structures then the $u$ component

In [ ]:
# plt.figure(8)
plt.subplots_adjust(right=0.99)

levels = np.linspace(-1, 1, 20)
skip = 2
scale = 10

for it in range(0, 100, 2):
    plt.clf()
    
    # plot reconstructed snapshot
    plt.subplot(211)
    up_r = np.reshape(Xp_r[:nx*ny, it], (nx, ny))
    vp_r = np.reshape(Xp_r[nx*ny:, it], (nx, ny))

    plt.contourf(mat["x"], mat["y"], vp_r, levels, cmap="seismic")
    plt.colorbar(shrink=0.6, label="$v^\prime_r(x, y, t)$", ticks=[-1, 0, 1])
    plt.gca().set_aspect(1)
    plt.title("Reconstructed")
    
    # the scale argument controls the length of the vector
    # (lower scale -> longer vectors)
    plt.quiver(mat["x"][::skip, ::skip], 
               mat["y"][::skip, ::skip],
               up_r[::skip, ::skip],
               vp_r[::skip, ::skip], scale=scale)

    plot_airfoil(AoA)
    plt.xlabel("x/c")
    plt.ylabel("y/c")
    plt.grid(1)
    
    # plot actual snapshot
    plt.subplot(212)
    up = np.reshape(Xp[:nx*ny, it], (nx, ny))
    vp = np.reshape(Xp[nx*ny:, it], (nx, ny))

    plt.contourf(mat["x"], mat["y"], vp, levels, cmap="seismic")
    plt.colorbar(shrink=0.6, label="$v^\prime(x, y, t)$", ticks=[-1, 0, 1])
    plt.gca().set_aspect(1)
    plt.title("Original")
    
    # the scale argument controls the length of the vector
    # (lower scale -> longer vectors)
    plt.quiver(mat["x"][::skip, ::skip], 
               mat["y"][::skip, ::skip],
               up[::skip, ::skip],
               vp[::skip, ::skip], scale=scale)

    plot_airfoil(AoA)
    plt.xlabel("x/c")
    plt.ylabel("y/c")
    plt.grid(1)
    
    plt.pause(0.001)

There are several remarks to be made:

- the low rank reconstructed velocity field is much less noisy. All the noise upstream the airfoil due to measurement error has been removed. In fact, POD (and the SVD) works well as a denoisin technique.

- the velocity fluctuations in the wake are more intense than what the reconstructed field predicts. This is normal, as with two modes we have only captured a fraction of the total energy of the fluctuations. 

- we have not really captured any of flow activity that features prominently in the near wake, e.g. the flow separation process starting at the airfoil's leading edge.

### Task 6 - other things to work on in your own time

Now, here's a couple of interesting experiments to try at home.

- try reconstructing the velocity fluctuations using only the first POD mode. You'll see that the ability of the low-rank reconstruction to model the convection of flow structures in the wake is lost. You need two modes shifted by half a period to describe this behaviour.

- try a higher-rank reconstruction, e.g. with 10 or 20 modes.

- explore the other dataset, for angle of attack $\alpha=0$. Look at the POD modes and the temporal coefficients, and compare the drop of the modal energies with the dataset we have used in this lab.

### Bonus task - Fourier analysis

We can investigate the frequency content of the temporal coefficient $a_1(t)$ by computing its Fourier transform, to extract the frequency of the oscillation we see in the plot. For this, we can use the Welch method available in the `scipy.signal` package.

In [ ]:
import scipy.signal as ssg

The function `welch` returns a 1D array of frequencies and a 1D array with the corresponding energy.

In [ ]:
plt.figure(9)
plt.clf()

# plot the spectra for these modes
idxs = [300]

plt.subplot(211)
for idx in idxs:
    # the sampling frequency is the inverse of the sampling interval
    f, P = ssg.welch(A[:, idx], fs=1/mat["dt"][0][0], window='hann', nperseg=1000)
    plt.loglog(f, P, label="i=%d" % idx)

plt.legend()
plt.grid(1)
plt.xlabel("$f c / u_\infty$")
plt.ylabel("PSD")


plt.subplot(212)
for idx in idxs:
    plt.plot(tt, A[:, idx], label="i=%d" % idx)

plt.legend()
plt.grid(1)
plt.xlabel("t")
plt.ylabel("$a_i(t)$")

Note the label of the horizontal axis: the frequency is non-dimensional and it is scaled with the airfol chord and free stream velocity. What is the name of this nondimensional number?

In any case, we see that the energy peaks at a normalised frequency of about 0.85, but the spectrum shows significant energy away from the peak too! This is the symptom that the first POD temporal coefficient also contains information about flow processes that occur at other frequencies. 